In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
import tensorflow as tf
import os
from os.path import isfile, join
import numpy as np
import shutil
from tensorflow import keras
from pathlib import Path
from IPython.display import display, Audio
import subprocess

In [5]:
#/kaggle/input/articleandtwentytwofivenewdata/archive
#!cp -r "../input/articleandtwentytwofivenewdata/archive" ./

In [6]:
data_directory = "archive/16000_pcm_speeches"
audio_folder = "audio"
noise_folder = "noise"

audio_path = os.path.join(data_directory, audio_folder)
noise_path = os.path.join(data_directory, noise_folder)

In [7]:
audio_path

'archive/16000_pcm_speeches\\audio'

In [8]:
valid_split = 0.1

shuffle_seed = 43

sample_rate = 16000

scale = 0.5

batch_size = 128

epochs = 15

In [9]:
# for folder in os.listdir(data_directory):
#     if os.path.isdir(os.path.join(data_directory, folder)):
#         if folder in [audio_folder, noise_folder]:
            
#             continue
#         elif folder in ["other", "_background_noise_"]:
            
#             shutil.move(
#                 os.path.join(data_directory, folder),
#                 os.path.join(noise_path, folder),
#             )
#         else:
#             shutil.move(
#                 os.path.join(data_directory, folder),
#                 os.path.join(audio_path, folder),
#             )

In [10]:
noise_paths = []
for subdir in os.listdir(noise_path):
    subdir_path = Path(noise_path) / subdir
    if os.path.isdir(subdir_path):
        noise_paths += [
            os.path.join(subdir_path, filepath)
            for filepath in os.listdir(subdir_path)
            if filepath.endswith(".wav")
        ]

In [11]:
noise_paths

['archive\\16000_pcm_speeches\\noise\\other\\exercise_bike.wav',
 'archive\\16000_pcm_speeches\\noise\\other\\pink_noise.wav',
 'archive\\16000_pcm_speeches\\noise\\_background_noise_\\10convert.com_Audience-Claps_daSG5fwdA7o.wav',
 'archive\\16000_pcm_speeches\\noise\\_background_noise_\\doing_the_dishes.wav',
 'archive\\16000_pcm_speeches\\noise\\_background_noise_\\dude_miaowing.wav',
 'archive\\16000_pcm_speeches\\noise\\_background_noise_\\running_tap.wav']

In [12]:
command = (
    "for dir in `ls -1 " + noise_path + "`; do "
    "for file in `ls -1 " + noise_path + "/$dir/*.wav`; do "
    "sample_rate=`ffprobe -hide_banner -loglevel panic -show_streams "
    "$file | grep sample_rate | cut -f2 -d=`; "
    "if [ $sample_rate -ne 16000 ]; then "
    "ffmpeg -hide_banner -loglevel panic -y "
    "-i $file -ar 16000 temp.wav; "
    "mv temp.wav $file; "
    "fi; done; done"
)

In [13]:
os.system(command)
def load_noise_sample(path):
    sample, sampling_rate = tf.audio.decode_wav(
        tf.io.read_file(path), desired_channels=1
    )
    if sampling_rate == sample_rate:
        slices = int(sample.shape[0] / sample_rate)
        sample = tf.split(sample[: slices * sample_rate], slices)
        return sample
    else:
        print("Sampling rate for",path, "is incorrect")
        return None


noises = []
for path in noise_paths:
    sample = load_noise_sample(path)
    if sample:
        noises.extend(sample)
noises = tf.stack(noises)

Sampling rate for archive\16000_pcm_speeches\noise\other\exercise_bike.wav is incorrect
Sampling rate for archive\16000_pcm_speeches\noise\other\pink_noise.wav is incorrect
Sampling rate for archive\16000_pcm_speeches\noise\_background_noise_\10convert.com_Audience-Claps_daSG5fwdA7o.wav is incorrect
Sampling rate for archive\16000_pcm_speeches\noise\_background_noise_\doing_the_dishes.wav is incorrect
Sampling rate for archive\16000_pcm_speeches\noise\_background_noise_\dude_miaowing.wav is incorrect
Sampling rate for archive\16000_pcm_speeches\noise\_background_noise_\running_tap.wav is incorrect


In [14]:
def paths_and_labels_to_dataset(audio_paths, labels):
    path_ds = tf.data.Dataset.from_tensor_slices(audio_paths)
    audio_ds = path_ds.map(lambda x: path_to_audio(x))
    label_ds = tf.data.Dataset.from_tensor_slices(labels)
    return tf.data.Dataset.zip((audio_ds, label_ds))

In [15]:
def path_to_audio(path):
    audio = tf.io.read_file(path)
    audio, _ = tf.audio.decode_wav(audio, 1, sample_rate)
    return audio

In [16]:
def add_noise(audio, noises=None, scale=0.5):
    if noises is not None:
        tf_rnd = tf.random.uniform(
            (tf.shape(audio)[0],), 0, noises.shape[0], dtype=tf.int32
        )
        noise = tf.gather(noises, tf_rnd, axis=0)

        prop = tf.math.reduce_max(audio, axis=1) / tf.math.reduce_max(noise, axis=1)
        prop = tf.repeat(tf.expand_dims(prop, axis=1), tf.shape(audio)[1], axis=1)

        audio = audio + noise * prop * scale

    return audio

In [17]:
def audio_to_fft(audio):
    audio = tf.squeeze(audio, axis=-1)
    fft = tf.signal.fft(
        tf.cast(tf.complex(real=audio, imag=tf.zeros_like(audio)), tf.complex64)
    )
    fft = tf.expand_dims(fft, axis=-1)

    return tf.math.abs(fft[:, : (audio.shape[1] // 2), :])

In [18]:
class_names = os.listdir(audio_path)
print(class_names,)

audio_paths = []
labels = []
for label, name in enumerate(class_names):
    print("Speaker:",(name))
    dir_path = Path(audio_path) / name
    speaker_sample_paths = [
        os.path.join(dir_path, filepath)
        for filepath in os.listdir(dir_path)
        if filepath.endswith(".wav")
    ]
    audio_paths += speaker_sample_paths
    labels += [label] * len(speaker_sample_paths)

['Abhishek', 'Arunanshu', 'Shivam', 'Sunamdha']
Speaker: Abhishek
Speaker: Arunanshu
Speaker: Shivam
Speaker: Sunamdha


In [19]:
rng = np.random.RandomState(shuffle_seed)
rng.shuffle(audio_paths)
rng = np.random.RandomState(shuffle_seed)
rng.shuffle(labels)

In [20]:
# Split into training and validation
num_val_samples = int(valid_split * len(audio_paths))
train_audio_paths = audio_paths[:-num_val_samples]
train_labels = labels[:-num_val_samples]


valid_audio_paths = audio_paths[-num_val_samples:]
valid_labels = labels[-num_val_samples:]

In [21]:
# Create datasets, one for training and the other for validation
train_ds = paths_and_labels_to_dataset(train_audio_paths, train_labels)
train_ds = train_ds.shuffle(buffer_size=batch_size * 8, seed=shuffle_seed).batch(
    batch_size
)

valid_ds = paths_and_labels_to_dataset(valid_audio_paths, valid_labels)
valid_ds = valid_ds.shuffle(buffer_size=32 * 8, seed=shuffle_seed).batch(32)

In [22]:
# Add noise to the training set
# train_ds = train_ds.map(
#     lambda x, y: (add_noise(x, noises, scale=scale), y),
#     num_parallel_calls=tf.data.experimental.AUTOTUNE,
# )

# Transform audio wave to the frequency domain using `audio_to_fft`
train_ds = train_ds.map(
    lambda x, y: (audio_to_fft(x), y), num_parallel_calls=tf.data.experimental.AUTOTUNE
)

train_ds = train_ds.prefetch(tf.data.experimental.AUTOTUNE)

valid_ds = valid_ds.map(
    lambda x, y: (audio_to_fft(x), y), num_parallel_calls=tf.data.experimental.AUTOTUNE
)
valid_ds = valid_ds.prefetch(tf.data.experimental.AUTOTUNE)

In [23]:
train_ds

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 8000, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [24]:
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import Dropout
import joblib

In [25]:
# def residual_block(x, filters, conv_num = 3, activation = "relu"):
#     s = keras.layers.Conv1D(filters, 1, padding = "same")(x)
    
#     for i in range(conv_num - 1):
#         x = keras.layers.Conv1D(filters, 3, padding = "same")(x)
#         x = keras.layers.Activation(activation)(x)
#         #x = Dropout(dropout_rate)(x)
        
    
#     x = keras.layers.Conv1D(filters, 3, padding = "same")(x)
#     x = keras.layers.Add()([x, s])
#     x = keras.layers.Activation(activation)(x)
#     #x = Dropout(dropout_rate)(x)
    
#     return keras.layers.MaxPool1D(pool_size = 2, strides = 2)(x)

# def build_model(input_shape, num_classes, dropout_rate=0.2):
#     inputs = keras.layers.Input(shape = input_shape, name = "input")
    
#     x = residual_block(inputs, 16, 2)
#     x = residual_block(inputs, 32, 2)
#     x = residual_block(inputs, 64, 3)
#     #x = residual_block(inputs, 128, 3)
#     x = residual_block(inputs, 128, 3)
#     x = keras.layers.AveragePooling1D(pool_size=3, strides=3)(x)
#     x = keras.layers.Flatten()(x)
#     x = keras.layers.Dense(256, activation="relu")(x)
#     x = Dropout(dropout_rate)(x)
#     x = keras.layers.Dense(128, activation="relu")(x)
#     x = keras.layers.Dropout(dropout_rate)(x)
    
#     outputs = keras.layers.Dense(num_classes, activation = "softmax", name = "output")(x)
    
#     return keras.models.Model(inputs = inputs, outputs = outputs)


# dropout_rate = 0.2
# model = build_model((sample_rate // 2, 1), len(class_names), dropout_rate=dropout_rate)

# model.summary()
# def lr_schedule(epoch):
#     initial_learning_rate = 0.001
#     decay = 0.9
#     lr = initial_learning_rate * decay ** epoch
#     return lr

# lr_scheduler = keras.callbacks.LearningRateScheduler(lr_schedule)


# model.compile(optimizer="Adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]) 

# model_save_filename = "/kaggle/working/archive/model/fivespeakertwentytofivemodel.joblib"

# earlystopping_cb = keras.callbacks.EarlyStopping(monitor='loss', patience=2, restore_best_weights=True)

# #mdlcheckpoint_cb = keras.callbacks.ModelCheckpoint(model_save_filename, monitor="loss", save_best_only=True)

In [26]:
import keras
from keras.layers import Dropout, Conv1D, Activation, Add, MaxPool1D, AveragePooling1D, Flatten, Dense, Input, BatchNormalization
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import numpy as np

def residual_block(x, filters, conv_num=3, activation="relu", use_batch_norm=False, l2_reg=0.01):
    s = Conv1D(filters, 1, padding="same", kernel_regularizer=keras.regularizers.l2(l2_reg))(x)

    for i in range(conv_num - 1):
        x = Conv1D(filters, 3, padding="same", kernel_regularizer=keras.regularizers.l2(l2_reg))(x)
        if use_batch_norm:
            x = BatchNormalization()(x)
        x = Activation(activation)(x)
        # x = Dropout(dropout_rate)(x)

    x = Conv1D(filters, 3, padding="same", kernel_regularizer=keras.regularizers.l2(l2_reg))(x)
    x = Add()([x, s])
    x = Activation(activation)(x)
    # x = Dropout(dropout_rate)(x)

    return MaxPool1D(pool_size=2, strides=2)(x)

def build_model(input_shape, num_classes, dropout_rate=0.2, use_batch_norm=False, l2_reg=0.01):
    inputs = Input(shape=input_shape, name="input")

    x = residual_block(inputs, 16, 2, use_batch_norm=use_batch_norm, l2_reg=l2_reg)
    x = residual_block(x, 32, 2, use_batch_norm=use_batch_norm, l2_reg=l2_reg)
    x = residual_block(x, 64, 3, use_batch_norm=use_batch_norm, l2_reg=l2_reg)
    # x = residual_block(inputs, 128, 3)
    x = residual_block(x, 128, 3, use_batch_norm=use_batch_norm, l2_reg=l2_reg)
    x = AveragePooling1D(pool_size=3, strides=3)(x)
    x = Flatten()(x)
    x = Dense(256, activation="relu")(x)
    x = Dropout(dropout_rate)(x)
    x = Dense(128, activation="relu")(x)
    x = Dropout(dropout_rate)(x)

    outputs = Dense(num_classes, activation="softmax", name="output")(x)

    return Model(inputs=inputs, outputs=outputs)

# Data loading and preprocessing
# Replace X_train, X_test, y_train, y_test with your actual data

# Initialize the model
dropout_rate = 0.2
use_batch_norm = True
l2_reg = 0.001  # Regularization strength
model = build_model((sample_rate // 2, 1), len(class_names), dropout_rate=dropout_rate, use_batch_norm=use_batch_norm, l2_reg=l2_reg)

# Define callbacks
early_stopping_cb = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
#model_save_filename = "/kaggle/working/archive/model/fivespeakertwentytofivemodel.h5"
#mdl_checkpoint_cb = ModelCheckpoint(model_save_filename, monitor="val_accuracy", save_best_only=True)
reduce_lr_cb = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)

# Compile the model
model.compile(optimizer="Adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Train the model
history = model.fit(train_ds, epochs=epochs, batch_size=32, validation_data=(valid_ds), callbacks=[early_stopping_cb,  reduce_lr_cb])





Epoch 1/15


34/34 [==============================] - 183s 4s/step - loss: 1.0239 - accuracy: 0.8984 - val_loss: 0.8038 - val_accuracy: 0.9560 - lr: 0.0010
Epoch 2/15
34/34 [==============================] - 136s 4s/step - loss: 0.6706 - accuracy: 0.9904 - val_loss: 0.6363 - val_accuracy: 0.9916 - lr: 0.0010
Epoch 3/15
34/34 [==============================] - 223s 6s/step - loss: 0.5959 - accuracy: 0.9923 - val_loss: 0.5612 - val_accuracy: 0.9916 - lr: 0.0010
Epoch 4/15
34/34 [==============================] - 227s 5s/step - loss: 0.5335 - accuracy: 0.9914 - val_loss: 0.5003 - val_accuracy: 0.9916 - lr: 0.0010
Epoch 5/15
34/34 [==============================] - 180s 4s/step - loss: 0.4711 - accuracy: 0.9925 - val_loss: 0.4531 - val_accuracy: 0.9895 - lr: 0.0010
Epoch 6/15
34/34 [==============================] - 137s 4s/step - loss: 0.4199 - accuracy: 0.9946 - val_loss: 0.4153 - val_accuracy: 0.9916 - lr: 0.0010
Epoch 7/15
34/34 [==============================] - 208s 5s/step - loss

In [27]:
# history = model.fit(
#     train_ds,
#     epochs=epochs,
#     validation_data=valid_ds,
#     callbacks=[earlystopping_cb],
# )

In [28]:
# Save the model using joblib
model_save_filename="UPlocal_four_speaker_model.joblib"
joblib.dump(model, model_save_filename)

['UPlocal_four_speaker_model.joblib']

In [29]:
print("Accuracy of model:",model.evaluate(valid_ds))

15/15 [==============================] - 12s 365ms/step - loss: 0.1688 - accuracy: 0.9979
Accuracy of model: [0.16879761219024658, 0.99790358543396]


In [30]:
SAMPLES_TO_DISPLAY = 10

test_ds = paths_and_labels_to_dataset(valid_audio_paths, valid_labels)
test_ds = test_ds.shuffle(buffer_size=batch_size * 8, seed=shuffle_seed).batch(
    batch_size
)

# test_ds = test_ds.map(lambda x, y: (add_noise(x, noises, scale=scale), y))

for audios, labels in test_ds.take(1):
    ffts = audio_to_fft(audios)
    y_pred = model.predict(ffts)
    rnd = np.random.randint(0, batch_size, SAMPLES_TO_DISPLAY)
    audios = audios.numpy()[rnd, :, :]
    labels = labels.numpy()[rnd]
    y_pred = np.argmax(y_pred, axis=-1)[rnd]

    for index in range(SAMPLES_TO_DISPLAY):
        print(
            "Speaker:\33{} {}\33[0m\tPredicted:\33{} {}\33[0m".format(
                "[92m" if labels[index] == y_pred[index] else "[91m",
                class_names[labels[index]],
                "[92m" if labels[index] == y_pred[index] else "[91m",
                class_names[y_pred[index]],
            )
        )
        if labels[index] ==y_pred[index]:
            print("Welcome")
        else:
            print("Sorry")
        print("The speaker is" if labels[index] == y_pred[index] else "", class_names[y_pred[index]])

4/4 [==============================] - 1s 103ms/step
Speaker: Abhishek	Predicted: Abhishek
Welcome
The speaker is Abhishek
Speaker: Shivam	Predicted: Shivam
Welcome
The speaker is Shivam
Speaker: Arunanshu	Predicted: Arunanshu
Welcome
The speaker is Arunanshu
Speaker: Arunanshu	Predicted: Arunanshu
Welcome
The speaker is Arunanshu
Speaker: Sunamdha	Predicted: Sunamdha
Welcome
The speaker is Sunamdha
Speaker: Sunamdha	Predicted: Sunamdha
Welcome
The speaker is Sunamdha
Speaker: Sunamdha	Predicted: Sunamdha
Welcome
The speaker is Sunamdha
Speaker: Sunamdha	Predicted: Sunamdha
Welcome
The speaker is Sunamdha
Speaker: Abhishek	Predicted: Abhishek
Welcome
The speaker is Abhishek
Speaker: Arunanshu	Predicted: Arunanshu
Welcome
The speaker is Arunanshu


**Test Data**

In [31]:
#/kaggle/input/articleandtwentytwofivenewdata/archive/Article/16000_pcm_speeches

data_directory = "archive/Article/16000_pcm_speeches"
audio_folder = "audio"
noise_folder = "noise"

audio_path = os.path.join(data_directory, audio_folder)
noise_path = os.path.join(data_directory, noise_folder)

In [32]:
audio_path

'archive/Article/16000_pcm_speeches\\audio'

In [33]:
# for folder in os.listdir(data_directory):
#     if os.path.isdir(os.path.join(data_directory, folder)):
#         if folder in [audio_folder, noise_folder]:
            
#             continue
#         elif folder in ["other", "_background_noise_"]:
            
#             shutil.move(
#                 os.path.join(data_directory, folder),
#                 os.path.join(noise_path, folder),
#             )
#         else:
#             shutil.move(
#                 os.path.join(data_directory, folder),
#                 os.path.join(audio_path, folder),
#             )

In [34]:
noise_paths = []
for subdir in os.listdir(noise_path):
    subdir_path = Path(noise_path) / subdir
    if os.path.isdir(subdir_path):
        noise_paths += [
            os.path.join(subdir_path, filepath)
            for filepath in os.listdir(subdir_path)
            if filepath.endswith(".wav")
        ]

In [35]:
noise_paths

['archive\\Article\\16000_pcm_speeches\\noise\\other\\exercise_bike.wav',
 'archive\\Article\\16000_pcm_speeches\\noise\\other\\pink_noise.wav',
 'archive\\Article\\16000_pcm_speeches\\noise\\_background_noise_\\10convert.com_Audience-Claps_daSG5fwdA7o.wav',
 'archive\\Article\\16000_pcm_speeches\\noise\\_background_noise_\\doing_the_dishes.wav',
 'archive\\Article\\16000_pcm_speeches\\noise\\_background_noise_\\dude_miaowing.wav',
 'archive\\Article\\16000_pcm_speeches\\noise\\_background_noise_\\running_tap.wav']

In [36]:
command = (
    "for dir in `ls -1 " + noise_path + "`; do "
    "for file in `ls -1 " + noise_path + "/$dir/*.wav`; do "
    "sample_rate=`ffprobe -hide_banner -loglevel panic -show_streams "
    "$file | grep sample_rate | cut -f2 -d=`; "
    "if [ $sample_rate -ne 16000 ]; then "
    "ffmpeg -hide_banner -loglevel panic -y "
    "-i $file -ar 16000 temp.wav; "
    "mv temp.wav $file; "
    "fi; done; done"
)

In [37]:
os.system(command)
def load_noise_sample(path):
    sample, sampling_rate = tf.audio.decode_wav(
        tf.io.read_file(path), desired_channels=1
    )
    if sampling_rate == sample_rate:
        slices = int(sample.shape[0] / sample_rate)
        sample = tf.split(sample[: slices * sample_rate], slices)
        return sample
    else:
        print("Sampling rate for",path, "is incorrect")
        return None


noises = []
for path in noise_paths:
    sample = load_noise_sample(path)
    if sample:
        noises.extend(sample)
noises = tf.stack(noises)

Sampling rate for archive\Article\16000_pcm_speeches\noise\other\exercise_bike.wav is incorrect
Sampling rate for archive\Article\16000_pcm_speeches\noise\other\pink_noise.wav is incorrect
Sampling rate for archive\Article\16000_pcm_speeches\noise\_background_noise_\10convert.com_Audience-Claps_daSG5fwdA7o.wav is incorrect
Sampling rate for archive\Article\16000_pcm_speeches\noise\_background_noise_\doing_the_dishes.wav is incorrect
Sampling rate for archive\Article\16000_pcm_speeches\noise\_background_noise_\dude_miaowing.wav is incorrect
Sampling rate for archive\Article\16000_pcm_speeches\noise\_background_noise_\running_tap.wav is incorrect


In [38]:
class_names = os.listdir(audio_path)
print(class_names,)

audio_paths = []
labels = []
for label, name in enumerate(class_names):
    print("Speaker:",(name))
    dir_path = Path(audio_path) / name
    speaker_sample_paths = [
        os.path.join(dir_path, filepath)
        for filepath in os.listdir(dir_path)
        if filepath.endswith(".wav")
    ]
    audio_paths += speaker_sample_paths
    labels += [label] * len(speaker_sample_paths)

['Abhishek', 'Arunanshu', 'Shivam', 'Sunamdha']
Speaker: Abhishek
Speaker: Arunanshu
Speaker: Shivam
Speaker: Sunamdha


In [39]:
# Shuffle to generate random data
rng = np.random.RandomState(shuffle_seed)
rng.shuffle(audio_paths)
rng = np.random.RandomState(shuffle_seed)
rng.shuffle(labels)

In [40]:
num_val_samples = int(valid_split * len(audio_paths))
test1_audio_paths = audio_paths[:-num_val_samples]
test1_labels = labels[:-num_val_samples]


test2_audio_paths = audio_paths[-num_val_samples:]
test2_labels = labels[-num_val_samples:]

In [41]:
# Create datasets, one for training and the other for validation
test1_ds = paths_and_labels_to_dataset(test1_audio_paths, test1_labels)
test1_ds = test1_ds.shuffle(buffer_size=batch_size * 8, seed=shuffle_seed).batch(
    batch_size
)

test2_ds = paths_and_labels_to_dataset(test2_audio_paths, test2_labels)
test2_ds = test2_ds.shuffle(buffer_size=32 * 8, seed=shuffle_seed).batch(32)

In [42]:
# Add noise to the training set
# test1_ds = test1_ds.map(
#     lambda x, y: (add_noise(x, noises, scale=scale), y),
#     num_parallel_calls=tf.data.experimental.AUTOTUNE,
# )

# Transform audio wave to the frequency domain using `audio_to_fft`
test1_ds = test1_ds.map(
    lambda x, y: (audio_to_fft(x), y), num_parallel_calls=tf.data.experimental.AUTOTUNE
)

test1_ds = test1_ds.prefetch(tf.data.experimental.AUTOTUNE)

test2_ds = test2_ds.map(
    lambda x, y: (audio_to_fft(x), y), num_parallel_calls=tf.data.experimental.AUTOTUNE
)
test2_ds = test2_ds.prefetch(tf.data.experimental.AUTOTUNE)

In [43]:
print("Accuracy of model:",model.evaluate(test1_ds))

9/9 [==============================] - 38s 581ms/step - loss: 0.4290 - accuracy: 0.9502
Accuracy of model: [0.4289529323577881, 0.9501748085021973]


In [44]:
print("Accuracy of model:",model.evaluate(test2_ds))

4/4 [==============================] - 5s 141ms/step - loss: 0.4239 - accuracy: 0.9606
Accuracy of model: [0.4238627254962921, 0.960629940032959]


In [45]:
def paths_to_dataset(audio_paths):
    path_ds = tf.data.Dataset.from_tensor_slices(audio_paths)
    return tf.data.Dataset.zip((path_ds))

def predict(path, labels):
    test = paths_and_labels_to_dataset(path, labels)


    test = test.shuffle(buffer_size=batch_size * 8, seed=shuffle_seed).batch(
    batch_size
    )
    test = test.prefetch(tf.data.experimental.AUTOTUNE)


    # test = test.map(lambda x, y: (add_noise(x, noises, scale=scale), y))

    for audios, labels in test.take(1):
        ffts = audio_to_fft(audios)
        y_pred = model.predict(ffts)
        rnd = np.random.randint(0, 1, 1)
        audios = audios.numpy()[rnd, :]
        labels = labels.numpy()[rnd]
        y_pred = np.argmax(y_pred, axis=-1)[rnd]

    
    
    for index in range(1):
        print(
            "Speaker:\33{} {}\33[0m\tPredicted:\33{} {}\33[0m".format(
                "[92m" if labels[0].decode('utf-8') == class_names[y_pred[0]] else "[91m",
                labels[0].decode('utf-8'),
                "[92m" if labels[0].decode('utf-8') == class_names[y_pred[0]] else "[91m",
                class_names[y_pred[0]],
            )
        )
        if labels[0].decode('utf-8') ==class_names[y_pred[0]]:
            print("Welcome")
        else:
            print("Sorry")
        print("The speaker is" if labels[0].decode('utf-8') == class_names[y_pred[0]] else "", class_names[y_pred[0]])

In [46]:


path = ["archive/Article/16000_pcm_speeches/audio/Abhishek/100.wav"]
labels = ["Abhishek"]
try:
    predict(path, labels)
except:
    print("Error! Check if the file correctly passed or not!")

1/1 [==============================] - 0s 426ms/step
Speaker: Abhishek	Predicted: Abhishek
Welcome
The speaker is Abhishek


In [47]:
path = ["/kaggle/working/archive/kaggletestdata/16000_pcm_speeches/audio/Arunanshu/153.wav"]
labels = ["unknown"]
try:
    predict(path, labels)
except:
    print("Error! Check if the file correctly passed or not!")

Error! Check if the file correctly passed or not!


In [48]:
path = ["/kaggle/working/archive/kaggletestdata/16000_pcm_speeches/audio/Sunamdha/350.wav"]
labels = ["unknown"]
try:
    predict(path, labels)
except:
    print("Error! Check if the file correctly passed or not!")

Error! Check if the file correctly passed or not!


In [49]:
path = ["/kaggle/working/archive/kaggletestdata/16000_pcm_speeches/audio/Shivam/53.wav"]
labels = ["unknown"]
try:
    predict(path, labels)
except:
    print("Error! Check if the file correctly passed or not!")

Error! Check if the file correctly passed or not!


In [50]:
path = ["/kaggle/working/archive/kaggletestdata/16000_pcm_speeches/audio/Abhishek/153.wav"]
labels = ["unknown"]
try:
    predict(path, labels)
except:
    print("Error! Check if the file correctly passed or not!")

Error! Check if the file correctly passed or not!


In [51]:
# import keras
# from keras.models import load_model

# # Load the saved model
# saved_model_path = "/kaggle/input/testingtwentytofivedata/archive/model/model.h5"
# loaded_model = load_model(saved_model_path)

# # Assuming you have your test data in test_ds
# # Evaluate the model on test data
# test_loss, test_accuracy = loaded_model.evaluate(train_ds)

# print("Test Loss:", test_loss)
# print("Test Accuracy:", test_accuracy)


In [52]:
# import joblib

# # Load the model
# loaded_model = joblib.load("/kaggle/working/archive/model/fivespeakertwentytofivemodel.joblib")

# # Make predictions on the training dataset
# predictions = loaded_model.predict(train_ds)

# # Assuming train_ds is a DataFrame, you can print the predictions
# print(predictions)


In [53]:
import joblib

# Load the model
loaded_model = joblib.load("UPlocal_four_speaker_model.joblib")

# Make predictions on the training dataset
test_loss, test_accuracy = loaded_model.evaluate(test1_ds)
# Assuming train_ds is a DataFrame, you can print the predictions



9/9 [==============================] - 16s 527ms/step - loss: 0.4290 - accuracy: 0.9502
